<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Pesquisa/3_laborat%C3%B3rio_de_varia%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importa a biblioteca glob para obter uma lista de caminhos de arquivos que correspondem a um determinado padrão.
import glob
# Importa a biblioteca OpenCV para processamento de imagem.
import cv2
# Importa a biblioteca NumPy para operações numéricas.
import numpy as np

from google.colab.patches import cv2_imshow

from google.colab import drive
drive.mount('/content/drive/')


%cd /content/drive/MyDrive/

Mounted at /content/drive/
/content/drive/MyDrive


In [ ]:
nomes = []
formas = []
areas = [] # Armazena a área de cada objeto
perimeters = [] # Armazena o perímetro de cada objeto
centroids_x = [] # Armazena as coordenadas x do centróide de cada objeto
centroids_y = [] # Armazena as coordenadas y do centróide de cada objeto
aspect_ratio = [] # Armazena a relação entre largura e altura do retângulo de contorno de cada objeto
extent = [] # Armazena a razão entre a área do contorno e a área do retângulo de contorno de cada objeto
solidity = [] # Armazena a razão entre a área do contorno e a área do casco convexo de cada objeto
equivalent_diameter = [] # Armazena o diâmetro equivalente (em pixels) do círculo com a mesma área de cada objeto

width = [] # Armazena a largura do retângulo de contorno de cada objeto
height = [] # Armazena a altura do retângulo de contorno de cada objeto
angle = [] # Armazena o ângulo (em graus) do retângulo de contorno de cada objeto
radius = [] # Armazena o raio do círculo de contorno de cada objeto

In [ ]:
frame = '/content/drive/MyDrive/Pesquisa_ISD_IC/Base de Dados/Dia5-TT-CCF473-Contexto1_frame0.jpg'

# Carrega a imagem especificada pelo caminho do arquivo.
im = cv2.imread(frame)

# Cria uma cópia da imagem original.
img_contours = im.copy()

# Converte a imagem para o espaço de cores em escala de cinza.
image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# Converte a imagem para o tipo float32.
image32f = np.float32(image)

# Aplica um filtro de média à imagem.
mu = cv2.blur(image32f, (3, 3))

# Aplica um filtro de variância à imagem.
mu2 = cv2.blur(image32f*image32f, (3, 3))
sigma = cv2.sqrt(mu2 - mu*mu)
sigma = sigma * 10
sigma = sigma.astype("uint8")

# Aplica um filtro de borramento com kernel de 5x5
sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

# Aplicando o filtro Laplaciano
laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

# Normalizando a saída para o intervalo de 0 a 255
sigma = cv2.convertScaleAbs(laplacian)

# Aplica uma limiarização com método de Otsu à imagem.
ret3,th3 = cv2.threshold(sigma,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Define o kernel para dilatação e erosão
kernel = np.ones((3,3), np.uint8)

# Dilata a imagem
img_dilated = cv2.dilate(th3, kernel, iterations=2)

# Erode a imagem
th3 = cv2.erode(img_dilated, kernel, iterations=2)

# Encontra os contornos na imagem limiarizada.
contours, hierarchy = cv2.findContours(th3, cv2.RETR_TREE, cv2.ADAPTIVE_THRESH_GAUSSIAN_C) # https://doi.org/10.1016/0734-189X(85)90016-7

# Loop sobre os contornos
for cnt in contours:

  # Aproxima o contorno por uma sequência de pontos
  approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

  # Calcula a área e o perímetro do contorno
  area = cv2.contourArea(cnt)
  perimeter = cv2.arcLength(cnt, True)

  # Verifica se o contorno é um círculo
  if perimeter > 0:
    circularity = 4 * np.pi * area / (perimeter ** 2)
  else:
    circularity = 0

  # Desenha um círculo
  if circularity > 0.39:

    forma = "circulo"

    (x, y), radius = cv2.minEnclosingCircle(cnt)

    # Limita o raio máximo do círculo
    max_radius = 100  # Define o raio máximo desejado
    radius = min(radius, max_radius)  # Limita o raio ao valor máximo

    center = (int(x), int(y))
    radius = int(radius)

    #if area > 2000 or circularity > 53:
    if area > 2000:
      if radius < 100 and radius > 20:
        cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
    
    else:
      print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))

  # Desenha um quadrado
  else:
    rect = cv2.minAreaRect(cnt)
    
    forma = "quadrado"
    radius = 0

    (x, y), (w, h), angle = rect

    if min(w, h) == 0:
      aspect_ratio = 0
    else:
      aspect_ratio = max(w, h) / min(w, h)
    

    if aspect_ratio < 2.1:
    #if aspect_ratio < 3:  
      # Desenha um quadrado
      box = cv2.boxPoints(rect)
      box = np.int0(box)

      if area > 65:
        cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 

    else:
        print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))

print()
print()
# Exibe a imagem pos-filtro.
cv2_imshow(mu)
# Exibe a imagem limiarizada.
cv2_imshow(th3)
# Exibe a imagem original copiada com os contornos encontrados.
cv2_imshow(img_contours)
print()
print()
print()